In [ ]:
from torch import float32, max, no_grad, save
from torch.nn import (
    Conv2d,
    CrossEntropyLoss,
    Flatten,
    Linear,
    MaxPool2d,
    Module,
    ReLU,
    Sequential,
    Softmax,
)
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR100
from torchvision.transforms.v2 import Compose, Grayscale, ToDtype, ToImage

In [ ]:
# Define the transformation to apply to the data
transform = Compose(
    [ToImage(), ToDtype(float32, scale=True), Grayscale(num_output_channels=1)]
)

# Load the CIFAR10 dataset
train_data = CIFAR100("cifar100_data", download=True, train=True, transform=transform)
test_data = CIFAR100("cifar100_data", download=True, train=False, transform=transform)

# Split the training set into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, valset = random_split(train_data, [train_size, val_size])

# Define the data loaders for the training, validation, and testing sets
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
class CNN(Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layers = Sequential(
            Conv2d(1, 32, kernel_size=3, padding="same", stride=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Conv2d(32, 64, kernel_size=3, padding="same", stride=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Flatten(),
            Linear(32 * 16 * 16, 128),
            ReLU(inplace=True),
            Linear(128, 100),
            Softmax(dim=1),
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
# Define hyperparameters
epochs = 10

# Initialize model and optimizer
model = CNN()
optimizer = Adam(model.parameters())

# Define loss function
criterion = CrossEntropyLoss()
# Train the model
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

save(model.state_dict(), "../../../Models/cifar100_cnn.pth")

In [ ]:
model.eval()

# Track test accuracy
correct = 0
total = 0

with no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(correct / total):.4f}")